# Model Evaluation
This notebook evaluates the trained ElasticNet model, logs metrics, and registers the model to MLflow.

In [155]:
import os

In [156]:
%pwd

'c:\\Users\\2021ICTS28\\Desktop\\red_wine_quality_prediction'

In [157]:
os.chdir(r"c:\Users\2021ICTS28\Desktop\red_wine_quality_prediction")

In [158]:
%pwd

'c:\\Users\\2021ICTS28\\Desktop\\red_wine_quality_prediction'

In [159]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/yshanuka123/red_wine_quality_prediction.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="yshanuka123"
os.environ["MLFLOW_TRACKING_PASSWORD"]="12aa8bae310f0795c7c23dcd43e1bb74fec38ea1"

In [160]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str

In [161]:
from redwine.constants import *
from redwine.utils.common import read_yaml, create_directories, save_json

In [162]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path = config.model_path,
            all_params=params,
            metric_file_name = config.metric_file_name,
            target_column = schema.name,
            mlflow_uri="https://dagshub.com/yshanuka123/red_wine_quality_prediction.mlflow",
           
        )

        return model_evaluation_config

In [163]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib

In [164]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    
    def eval_metrics(self,actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2
    


    def log_into_mlflow(self):
        
        # Set MLflow tracking URI
        mlflow.set_tracking_uri(self.config.mlflow_uri)
        print(f"MLflow tracking URI set to: {self.config.mlflow_uri}")
        
        # Set experiment name
        mlflow.set_experiment("red_wine_quality_evaluation")
        print("Experiment set to: red_wine_quality_evaluation")
        
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]

        predicted_qualities = model.predict(test_x)
        (rmse, mae, r2) = self.eval_metrics(test_y, predicted_qualities)
        
        # Saving metrics as local
        scores = {"rmse": rmse, "mae": mae, "r2": r2}
        save_json(path=Path(self.config.metric_file_name), data=scores)
        print(f"Metrics saved locally to: {self.config.metric_file_name}")
        
        print(f"RMSE: {rmse:.4f}, MAE: {mae:.4f}, R2: {r2:.4f}")

        try:
            with mlflow.start_run(run_name="model_evaluation_run"):
                print("MLflow run started successfully")
                
                # Log parameters as strings to avoid INVALID_PARAMETER_VALUE error
                params = {}
                for key, value in self.config.all_params.items():
                    params[str(key)] = str(value)
                
                mlflow.log_params(params)
                print(f"Parameters logged to MLflow: {params}")

                # Log metrics
                mlflow.log_metric("rmse", float(rmse))
                mlflow.log_metric("mae", float(mae))
                mlflow.log_metric("r2", float(r2))
                print("Metrics logged to MLflow")

                # Log model
                mlflow.sklearn.log_model(model, "model")
                print("Model logged to MLflow")
                
                print("✓ MLflow logging completed successfully!")
                
        except Exception as e:
            print(f"❌ Error logging to MLflow: {e}")
            print("Metrics have been saved locally in: artifacts/model_evaluation/metrics.json")

In [165]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(config=model_evaluation_config)
    model_evaluation.log_into_mlflow()
except Exception as e:
    raise e

[2025-12-17 18:05:11,712: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-12-17 18:05:11,713: INFO: common: yaml file: params.yaml loaded successfully]
[2025-12-17 18:05:11,714: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-12-17 18:05:11,715: INFO: common: created directory at: artifacts]
[2025-12-17 18:05:11,715: INFO: common: created directory at: artifacts/model_evaluation]
MLflow tracking URI set to: https://dagshub.com/yshanuka123/red_wine_quality_prediction.mlflow
Experiment set to: red_wine_quality_evaluation
[2025-12-17 18:05:12,860: INFO: common: json file saved at: artifacts\model_evaluation\metrics.json]
Metrics saved locally to: artifacts/model_evaluation/metrics.json
RMSE: 0.7174, MAE: 0.5633, R2: 0.2778
MLflow run started successfully
Parameters logged to MLflow: {'alpha': '0.2', 'l1_ratio': '0.1'}


2025/12/17 18:05:16 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


Metrics logged to MLflow
Model logged to MLflow
✓ MLflow logging completed successfully!
🏃 View run model_evaluation_run at: https://dagshub.com/yshanuka123/red_wine_quality_prediction.mlflow/#/experiments/1/runs/e6a78032973b436b8b79e9fd849a9205
🧪 View experiment at: https://dagshub.com/yshanuka123/red_wine_quality_prediction.mlflow/#/experiments/1
